In [ ]:
import ipywidgets as widgets
from IPython.display import display
from nmf_vis.scatter import create_scatterplot
from nmf_vis.heatmap import create_heatmap

In [ ]:
heatmap_output = widgets.Output()
scatter_output = widgets.Output()
sort_dropdown = widgets.Dropdown(
    options=["component", "alphabetical", "cancer_type", "organ_system"],
    value="component",
    description="Sort by:",
)

heatmap_widget = create_heatmap(cfg_path="conf/config.json", sort_method=sort_dropdown.value)
scatter_widget = create_scatterplot(cfg_path="conf/config.json", sort_method=sort_dropdown.value)
selection = None


def on_sort_change(change):
    with heatmap_output:
        heatmap_output.clear_output(wait=True)
        display(create_heatmap(cfg_path="conf/config.json", sort_method=change.new, sample_ids=selection))


sort_dropdown.observe(on_sort_change, names="value")


def on_selection_change(change):
    """Handles selection changes in the UMAP plot."""
    with heatmap_output:
        heatmap_output.clear_output(wait=True)
        global selection
        if change.new.size:
            selection = change.new
            display(create_heatmap(cfg_path="conf/config.json", sort_method=sort_dropdown.value, sample_ids=change.new))
        else:
            selection = None 
            display(create_heatmap(cfg_path="conf/config.json", sort_method=sort_dropdown.value))
             

scatter = scatter_widget.widget
scatter.observe(on_selection_change, names="selection")

# --- Initial display ---
with heatmap_output:
    display(heatmap_widget)

with scatter_output:
    display(scatter.show())

display(widgets.HBox([widgets.VBox([scatter_output, sort_dropdown]), heatmap_output]))

In [ ]:
scatter.data.iloc[selection]

In [ ]:
import polars as pl

df = pl.scan_parquet("https://projects.abdenlab.org/itcr/tcga-db/files.parquet")

In [ ]:
files = df.explode(
    "case_submitter_ids"
).filter(
    pl.col("case_submitter_ids").is_in(["TCGA-A7-A0D9"])
).collect()
files

In [ ]:
file_id = files[2]['id'].item()
index_file_id = files[2]['index_file_ids'].explode().first()

In [ ]:
from fsspec.implementations.cached import CachingFileSystem
import oxbow as ox
import fsspec

httpfs = fsspec.implementations.http.HTTPFileSystem(
    client_kwargs={
        "headers": {
            "X-Auth-Token": open(".env/gdc_token.txt").read().strip(),
        }
    }
)
fs = CachingFileSystem(fs=httpfs)

In [ ]:
ds = ox.from_vcf(
    lambda : fs.open(f"https://api.gdc.cancer.gov/data/{file_id}", "rb"),
    index=lambda : fs.open(f"https://api.gdc.cancer.gov/data/{index_file_id}", "rb"),
    compression="bgzf",
    info_fields=[],
    batch_size=1024
).regions(
    "chrX"
)